In [2]:
import pandas as pd
from pymongo import MongoClient, GEO2D
import pymongo
import json
import datetime
from bson.son import SON
import folium 
from math import *
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
import plotly.plotly as py
from plotly.graph_objs import *


In [3]:
df_20150810 = pd.read_csv('/home/kathrin/Doctorado/bus_rio/Datos_Omnibus/morte_motoqueiro_20150817/20150810.txt', delimiter = ('\t'))


In [4]:
df_20150810_line_317 = df_20150810[(df_20150810['line']=='317')]


In [5]:
#Estableciendo la conexion con la base de datos morte_motoquiro en MongoDB
db = MongoClient().morte_motoquiro_event

In [6]:
onibus317_shapes = pd.read_csv('/home/kathrin/Doctorado/bus_rio/Datos_Omnibus/Onibus/Shapes/gtfs_linha317-shapes.csv')

In [7]:
df_initial_point = onibus317_shapes[(onibus317_shapes['sequencia']==0)]
df_initial_point

,linha,descricao,agencia,sequencia,shape_id,latitude,longitude
0,317,317-CENTRAL X ALVORADA (VIA TUNEL SANTA BARBAR...,Fetranspor,0,17424059,-22.9049,-43.1917
1041,317,317-CENTRAL X ALVORADA (VIA TUNEL SANTA BARBAR...,Fetranspor,0,17589381,-23.0012,-43.3649


In [8]:
#Punto inicial del omnibus 317 Central do Brasil
initial_point = (-22.9049,-43.1917)
#Punto inicial del omnibus invertido
inverted_initial_point = (-43.1917, -22.9049)
#Punto final del omnibus 317 Centro
final_point = (-23.0012, -43.3649)
#Punto final del omnibus invertido
inverted_final_point = (-43.3649, -23.0012)

In [9]:
"""Funciones para renderizar el mapa en el html"""
from IPython.display import HTML
 
def inline_map(map):
    """
    Embeds the HTML source of the map directly into the IPython notebook.
    
    This method will not work if the map depends on any files (json data). Also this uses
    the HTML5 srcdoc attribute, which may not be supported in all browsers.
    """
    map._build_map()
    return HTML('<iframe srcdoc="{srcdoc}" style="width: 100%; height: 510px; border: none"></iframe>'.format(srcdoc=map.HTML.replace('"', '&quot;')))
 
def embed_map(map, path="map.html"):
    """
    Embeds a linked iframe to the map into the IPython notebook.
    
    Note: this method will not capture the source of the map into the notebook.
    This method should work for all maps (as long as they use relative urls).
    """
    map.create_map(path=path)
    return HTML('<iframe src="files/{path}" style="width: 100%; height: 510px; border: none"></iframe>'.format(path=path))

In [10]:
fmap317=folium.Map(location=[-22.923714, -43.258999], zoom_start=11)
locations317 = []
for row in onibus317_shapes.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    locations317.insert(len(locations317), [ row[1]['latitude'], row[1]['longitude'] ])
    
#Pintando Circulos de radio de 50m alrededor de los puntos incial y final 
fmap317.simple_marker(initial_point, popup='Central')
fmap317.simple_marker(final_point, popup='Alborada')
fmap317.simple_marker((-22.992551 , -43.249098), popup='Morte Motoqueiro')

#Punto inicial
fmap317.circle_marker(location=final_point, radius=50,line_color='green',
                 fill_color='#3186cc', fill_opacity=0.2)

#Punto inicial
fmap317.circle_marker(location=initial_point, radius=50,line_color='red',
                 fill_color='#3186cc', fill_opacity=0.2)

fmap317.line(locations317)
embed_map(fmap317)

In [11]:
#esto es para graficar todos los puntos de todos los omnibus 317 que cirsularon ese dia
for row in df_20150810_line_317.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    latlon = [ row[1]['latitude'], row[1]['longitude'] ]
    fmap317.polygon_marker( latlon, fill_color='green', num_sides=4, radius=6)
    

embed_map(fmap317)

In [12]:
#Function to compute the bus_ids circulating during a day  
bus_id_20150810_317 = db.dados20150810_line_317.distinct("bus_id")
bus_id_20150810_317


[u'C41404', u'C41410', u'C41409', u'C41401', u'C41407']

In [13]:
len(bus_id_20150810_317)

5

In [14]:
#meter en una lista todos los bus_id
def get_busId_list(busId_list):
    array_busId = []
    for bus in busId_list:
        strbus = str(bus)
        array_busId.insert(len(array_busId), strbus)
    return array_busId

In [16]:
array_busId = get_busId_list(bus_id_20150810_317)
array_busId

['C41404', 'C41410', 'C41409', 'C41401', 'C41407']

In [24]:
#funcion que dado un el bus_id de un omnibus y un punto inicial/final devuelve los puntos mas cercanos a ese punto. 
# A esa función se le debería pasar también la BD, pero ahora no se va a hacer asi
def near_points_to_origin (bus_id, inverted_origin_point):
    near_initial_points_array = []
    #la funcion $within recibe las coordenadas en orden inverso (primero longitud y después latitud)
    for row in db.dados20150810_line_317.find({"latlon": {"$within": {"$center": [inverted_origin_point, 0.0035499543]}}, "bus_id": bus_id}).sort("timestamp",pymongo.DESCENDING):
    #for row in db.dados20150817_line_317.find({"latlon": {"$within": {"$center": [inverted_origin_point, 0.0115499543]}}, "bus_id": bus_id}).sort("timestamp",pymongo.DESCENDING):   
        near_initial_points_array.insert(0, [row[u'timestamp'], row[u'latitude'], row[u'longitude'], row[u'speed']])
    #convert the array in panda dataframe 
    sorted_near_initial_points = pd.DataFrame(near_initial_points_array, columns=['timestamp', 'latitude', 'longitude', 'speed'])
    return sorted_near_initial_points

In [18]:
def bus_departure_hour (initial_points_timestamp_list):
    list_departure_hours = []
    i = 0
    if len(initial_points_timestamp_list) >1:
        for i in range(0, len(initial_points_timestamp_list)-1):
            resto = datetime.datetime.strptime(initial_points_timestamp_list[i+1], "%d-%m-%Y %H:%M:%S")-datetime.datetime.strptime(initial_points_timestamp_list[i], "%d-%m-%Y %H:%M:%S")
            resto_total = resto.total_seconds()/60
            #si la diferencia entre los timestamp del omnibus en el putno supera los 40 minutos es q dio un viaje y regreso, sino lo supera es que se mantuvo ese tiempo esperando en el punto
            if (resto_total > 40):
                 list_departure_hours.insert(len( list_departure_hours),datetime.datetime.strptime(initial_points_timestamp_list[i], "%d-%m-%Y %H:%M:%S"))
        #siempre es necesario insertar el ultimo valor de la lista de "initial_points_timestamp" en la lista de  list_departure_hours 
        list_departure_hours.insert(len( list_departure_hours),datetime.datetime.strptime(initial_points_timestamp_list[i+1], "%d-%m-%Y %H:%M:%S"))
    elif len(initial_points_timestamp_list) == 1:
        list_departure_hours.insert(len( list_departure_hours),datetime.datetime.strptime(initial_points_timestamp_list[0], "%d-%m-%Y %H:%M:%S"))
    else:
        list_departure_hours = []
    return list_departure_hours

In [19]:
#Funcion que retorna el arreglo con las horas de llegada del omnibus al punto inicial/final del trayecto
def bus_arrive_hour (initial_points_timestamp_list):
    list_arrive_hours = []
    #siempre es necesario insertar el ultimo valor de la lista de "initial_points_timestamp" en la lista de leblon_departure_hours 
    if len(initial_points_timestamp_list) >1:
        list_arrive_hours.insert(len(list_arrive_hours),datetime.datetime.strptime(initial_points_timestamp_list[0], "%d-%m-%Y %H:%M:%S"))
        for i in range(0,  len(initial_points_timestamp_list)-1):
            resto = datetime.datetime.strptime(initial_points_timestamp_list[i+1], "%d-%m-%Y %H:%M:%S")-datetime.datetime.strptime(initial_points_timestamp_list[i], "%d-%m-%Y %H:%M:%S")
            resto_total = resto.total_seconds()/60
            if (resto_total > 40):
                list_arrive_hours.insert(len(list_arrive_hours),datetime.datetime.strptime(initial_points_timestamp_list[i+1], "%d-%m-%Y %H:%M:%S"))
    elif (len(initial_points_timestamp_list) == 1):
        list_arrive_hours.insert(len(list_arrive_hours),datetime.datetime.strptime(initial_points_timestamp_list[0], "%d-%m-%Y %H:%M:%S"))
    else:
        list_arrive_hours = []        
    return list_arrive_hours

In [20]:
#Para convertir de formato timedelta para horas y minutos
def days_hours_minutes(travel_time):
    hours = travel_time.seconds//3600 
    minutes = (travel_time.seconds//60)%60
    #print '%d hours, %d minutes' % (hours,minutes)
    if minutes < 10:
        minutes = str(0)+str(minutes)
    return str(hours) + ":"+ str(minutes)

In [21]:
#Funcion que crea la tabla de viajes con los tiempos de viaje
def create_travel_table (name_A, arrive_array_A, departure_array_A, name_B, arrive_array_B, departure_array_B, bus_Id, bus_line):
    
    my_time_table = []   
    num_travel = 0    
    i = 0 #Cursor para moverme por el departure_array_A(arreglo q contiene las horas de salida del punto incial)   
    j = 0 #Cursor para moverme por el departure_array_B (arreglo q contiene las horas de salida del punto final)
   
    while (i<len(departure_array_A) and j< len(departure_array_B)):
        if (departure_array_A[i] < departure_array_B[j]):
            if(i+1 == len(departure_array_A)):
                direction = name_A + "_to_" + name_B
                departure_hour = departure_array_A[i]
                i = i +1
                arrive_hour = arrive_array_B[j]
                num_travel = num_travel +1
                travel_time = days_hours_minutes(arrive_hour - departure_hour)          
                my_time_table.insert(len(my_time_table), [bus_line, bus_Id, direction, departure_hour,arrive_hour, travel_time])
         
            elif ((i+1 < len(departure_array_A)) and (departure_array_B[j] < departure_array_A[i+1])):
                direction = name_A + "_to_" + name_B
                departure_hour = departure_array_A[i]
                i = i +1
                arrive_hour = arrive_array_B[j]
                num_travel = num_travel +1
                travel_time = days_hours_minutes(arrive_hour - departure_hour)          
                my_time_table.insert(len(my_time_table), [bus_line, bus_Id, direction, departure_hour,arrive_hour, travel_time])
            else: 
                i = i +1
            
        elif(departure_array_B[j] < departure_array_A[i]):
            if (j+1 == len(departure_array_B)):
                direction = name_B + "_to_" + name_A
                departure_hour = departure_array_B[j]
                j = j +1
                arrive_hour = arrive_array_A[i]
                num_travel = num_travel +1
                travel_time = days_hours_minutes(arrive_hour - departure_hour)          
                my_time_table.insert(len(my_time_table), [bus_line, bus_Id, direction, departure_hour,arrive_hour, travel_time])
            
            elif ((j+1 < len(departure_array_B)) and (departure_array_A[i] < departure_array_B[j+1])):
                direction = name_B + "_to_" + name_A
                departure_hour = departure_array_B[j]
                j = j +1
                arrive_hour = arrive_array_A[i]
                num_travel = num_travel +1
                travel_time = days_hours_minutes(arrive_hour - departure_hour)          
                my_time_table.insert(len(my_time_table), [bus_line, bus_Id, direction, departure_hour,arrive_hour, travel_time])
            else:
                j = j +1                
            
    #Construir el dataframe pasandole el array con todos los datos de los viajes   
    mydf = pd.DataFrame(my_time_table, columns=['Line', 'Bus_Id','Direction','Departure_Hour', 'Arrive_Hour', 'Travel_Time'])       
    return mydf  

In [22]:
#Funcion que crea la tabla de tiempo de viaje para cada bus_id de los que transitaron el el dia
def all_travel_table (line, punto_ini_coord_invertidas, name_punto_inicio, punto_fin_coord_invertidas, name_punto_final, array_busId):
    new_df = pd.DataFrame(columns=['Line', 'Bus_Id','Direction','Departure_Hour', 'Arrive_Hour', 'Travel_Time'])  
    for busId in array_busId:
        sorted_near_initial_points1 = near_points_to_origin(busId, punto_ini_coord_invertidas)
        sorted_near_final_points1 = near_points_to_origin(busId, punto_fin_coord_invertidas)
        initial_points_timestamp1 = sorted_near_initial_points1 ['timestamp']
        final_points_timestamp1 = sorted_near_final_points1 ['timestamp']
        #chequear si la lista initial_points_timestamp es diferente de vacio
        if len(initial_points_timestamp1) and len(final_points_timestamp1):
            vilaIsabel_departure_hours1 = bus_departure_hour(initial_points_timestamp1)
            first_point_departure_hour = bus_departure_hour (initial_points_timestamp1)
            last_point_departure_hour = bus_departure_hour (final_points_timestamp1)
            first_point_arrive_hour = bus_arrive_hour(initial_points_timestamp1)
            last_point_arrive_hour = bus_arrive_hour(final_points_timestamp1)
            my_time_table = create_travel_table(name_punto_inicio, first_point_arrive_hour, first_point_departure_hour, name_punto_final, last_point_arrive_hour, last_point_departure_hour, busId, line)
            new_df = new_df.append(my_time_table, ignore_index=True)
    return new_df

In [40]:
travel_table_20150810 = all_travel_table ('317', inverted_initial_point, 'Central', inverted_final_point, 'Alborada', array_busId)
travel_table_20150810

,Line,Bus_Id,Direction,Departure_Hour,Arrive_Hour,Travel_Time
0,317,C41404,Alborada_to_Central,2015-08-10 08:43:02,2015-08-10 10:25:17,1:42
1,317,C41404,Central_to_Alborada,2015-08-10 16:47:33,2015-08-10 19:01:08,2:13
2,317,C41404,Alborada_to_Central,2015-08-10 19:34:58,2015-08-10 20:50:09,1:15
3,317,C41410,Central_to_Alborada,2015-08-10 07:41:51,2015-08-10 09:14:08,1:32
4,317,C41410,Alborada_to_Central,2015-08-10 09:36:09,2015-08-10 11:09:11,1:33
5,317,C41410,Central_to_Alborada,2015-08-10 11:35:32,2015-08-10 13:23:18,1:47
6,317,C41410,Alborada_to_Central,2015-08-10 13:30:52,2015-08-10 14:46:22,1:15
7,317,C41410,Central_to_Alborada,2015-08-10 15:01:32,2015-08-10 16:30:07,1:28
8,317,C41410,Alborada_to_Central,2015-08-10 16:52:10,2015-08-10 18:35:44,1:43
9,317,C41409,Central_to_Alborada,2015-08-10 08:31:08,2015-08-10 09:45:03,1:13


In [41]:
sorted_travel_table_20150810 = travel_table_20150810.sort(["Departure_Hour"])
sorted_travel_table_20150810
interest_table_20150810 = sorted_travel_table_20150810[(sorted_travel_table_20150810["Direction"] == 'Central_to_Alborada')]
interest_table_20150810

,Line,Bus_Id,Direction,Departure_Hour,Arrive_Hour,Travel_Time
3,317,C41410,Central_to_Alborada,2015-08-10 07:41:51,2015-08-10 09:14:08,1:32
9,317,C41409,Central_to_Alborada,2015-08-10 08:31:08,2015-08-10 09:45:03,1:13
15,317,C41401,Central_to_Alborada,2015-08-10 09:09:15,2015-08-10 10:23:18,1:14
22,317,C41407,Central_to_Alborada,2015-08-10 10:32:20,2015-08-10 12:04:12,1:31
5,317,C41410,Central_to_Alborada,2015-08-10 11:35:32,2015-08-10 13:23:18,1:47
11,317,C41409,Central_to_Alborada,2015-08-10 12:08:09,2015-08-10 13:55:10,1:47
17,317,C41401,Central_to_Alborada,2015-08-10 13:10:53,2015-08-10 14:33:22,1:22
24,317,C41407,Central_to_Alborada,2015-08-10 13:39:11,2015-08-10 15:09:00,1:29
7,317,C41410,Central_to_Alborada,2015-08-10 15:01:32,2015-08-10 16:30:07,1:28
13,317,C41409,Central_to_Alborada,2015-08-10 15:56:34,2015-08-10 17:50:56,1:54


In [30]:
#esto es para graficar todos los puntos de todos los omnibus 317 que cirsularon ese dia
fmap317_C41404=folium.Map(location=[-22.923714, -43.258999], zoom_start=11)
locations317 = []
for row in onibus317_shapes.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    locations317.insert(len(locations317), [ row[1]['latitude'], row[1]['longitude'] ])
    
#Pintando Circulos de radio de 50m alrededor de los puntos incial y final 
#fmap317_C41130.simple_marker(initial_point, popup='Central')
fmap317_C41404.simple_marker(final_point, popup='Alborada')
fmap317_C41404.simple_marker((-22.992551 , -43.249098), popup='Morte Motoqueiro')

#Punto inicial
fmap317_C41404.circle_marker(location=final_point, radius=50,line_color='green',
                 fill_color='#3186cc', fill_opacity=0.2)

#Punto inicial
fmap317_C41404.circle_marker(location=initial_point, radius=50,line_color='red',
                 fill_color='#3186cc', fill_opacity=0.2)

fmap317_C41404.line(locations317)

df_20150810_line_317_C41404 = df_20150810_line_317[(df_20150810_line_317['bus_id']=='C41404')]
for row in df_20150810_line_317_C41404.iterrows():
    #The 'latitude' and 'longitude' names correspond to column names in the original cceats dataframe
    latlon = [ row[1]['latitude'], row[1]['longitude'] ]
    fmap317_C41404.polygon_marker( latlon, fill_color='green', num_sides=4, radius=6)
    

embed_map(fmap317_C41404)

In [32]:
#Funcion para calcular la distancia entre 2 puntos, teniendo en cuenta el radio de la tierra
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return (c * r)*1000 # return the distance in m

In [33]:
"""
funcion que dado un vector de lat, long y un punto A(lat,long), me devuelve un vector con
la distancia de cada par al punto A
 """
def vector_distancia(vector, point):
    vector_distance = []
    lat2 = point[0]
    long2 = point[1]
    for item in vector:
        lat1 = item[1]
        long1 = item[2]
        distance = haversine(long2, lat2, long1, lat1)
        latlon = (item[1], item[2])
        vector_distance.insert(len(vector_distance), [item[0],latlon, distance])
    df_vector_distance = pd.DataFrame(vector_distance, columns=['timestamp', 'latlon', 'distance'])
    return df_vector_distance   

In [34]:

#para seleccionar solo algunas columnas del dataframe, en este caso la columna 1(timestamp), la 4(latitude) y la 5(longitude)
important_columns_C41404 = df_20150810_line_317_C41404.iloc[:,[0,3,4]]
vector_C41404 = important_columns_C41404.values.tolist()

vector_dist_to_ini_point_C41404 = vector_distancia(vector_C41404, initial_point)
sorted_vector_dist_to_ini_point_C41404 = vector_dist_to_ini_point_C41404.sort(["timestamp"])

vector_dist_to_final_point_C41404  = vector_distancia(vector_C41404, final_point)
sorted_vector_dist_to_final_point_C41404 = vector_dist_to_final_point_C41404.sort(["timestamp"])

In [35]:
#hacer una grafica interactiva con la libreria de python (plotly.plotly). Se instala por consola asi: sudo pip install plotly
#distancia del punto final (central)

trace_C41404 = Scatter(x = sorted_vector_dist_to_ini_point_C41404['timestamp'], y = sorted_vector_dist_to_ini_point_C41404['distance'])
data_C41404 = Data([trace_C41404])
py.iplot(data_C41404, filename='pandas-time-series')

In [36]:
#hacer una grafica interactiva con la libreria de python (plotly.plotly). Se instala por consola asi: sudo pip install plotly
#distancia del punto final (central)

trace_C41404 = Scatter(x = sorted_vector_dist_to_final_point_C41404['timestamp'], y = sorted_vector_dist_to_final_point_C41404['distance'])
data_C41404 = Data([trace_C41404])
py.iplot(data_C41404, filename='pandas-time-series')